## Shamirs Secret Sharing

[Wikipedia link](https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing)

Wiki: Shamir's secret sharing (SSS) is an efficient secret sharing algorithm for distributing private information (the "secret") among a group so that the secret cannot be revealed unless a quorum of the group acts together to pool their knowledge.

In short it works by deciding on a secret (i.e. a number), lets call it $S$, and defining a polynomial of some degree $y$ s.t. $y(0) = S$. The degree of the polynomial will decide how many participants are needed to recover the secret, for example if three are needed then $y$ needs to be of degree two. Then a number of points along the curve is generated and they can be used to determine the curve, and thus be traced back to the secret.

Step 1 - Imports

In [19]:
import random
from sympy import poly
from sympy.abc import x
from collections import deque

Step 2 - Define the function to generate secret and its shares

We define the generating function, which distributes a number of secret-shares or points in the plane. In this example we decide that to recover the secret we need to have three participants. This means that the function will generate a polynomial y of degree two:

$$y(x) = ax^2 + bx + c$$

It will then produce $p$ points along the polynomial. In the example below we set $p=5$. This means that 3 of the total 5 can collaborate to reconstruct the polynomial, since there is only one such curve that passes through all points. For a simpler example, imagine the polynomial being a line, $y(x) = ax + b$. Then only two participants is needed to reconstruct the polynomial, since for a pair of points there exists one unique line that passes through it.

In [20]:
# small second degree example of Shamirs secret sharing
def shamir_generate(p, s):  # p = participants, secret

    # start by generating 2nd degree polynomial with f(0) = s
    a, b = random.sample(range(1, 100), 2)
    y = poly(a * (x ** 2) + b * x + s)

    # generate p points that lies on the polynomial
    points = []
    for i in range(p):
        xi = random.randint(1, 100)  # needs to not pick same x multiple times
        yi = y(xi)

        points.append((xi, yi))

    # print(y)

    # return and distribute a point to every participant
    print("secret polynomial:", y)
    print("secret:", s)
    print("points:", points)
    return points

Step 3 - Define the reconstructing function

The hard part of the algorithm is recovering the secret. It relies on so-called [Lagrange Polynomials](https://en.wikipedia.org/wiki/Lagrange_polynomial) to determine the polynomial from the shares.

In short, let the points used to reconstruct the curve be $P = p_1, \dots , p_n = (x_1, y_1), \dots, (x_n, y_n)$

Then the polynomial can be defined as:

$$f(x) = \sum_{i \in n} y_i \cdot \delta_i(x)$$

Now, since we are only really interested in $f(0)$, we only need to compute $\delta_i(0)$ at each iteration.

With this in mind we utilize the `lin_comb(l)` function to generate pairs $(a,b)$ from the x-coordinates, $(a,b) \in X^2, X = x_1, \dots x_n$:

$$\delta_i(0) = \frac{-a}{i-a} \cdot \frac{-b}{i-b} $$

Finally, we will get the secret at $f(0)$.

We start by defining a helper function and then the reconstruction function:

In [21]:
# produces a list of linear combinations of a list
# ie every possible pair (x,y) where x,y are elements in l, note (x,y) = (y,x)
def lin_comb(l):
    pairs = []

    for i in range(len(l)):
        if i + 1 >= len(l):
            pairs.append((l[0], l[-1]))
        else:
            pairs.append((l[i], l[i + 1]))
    
    return pairs



In [22]:
def shamir_reconstruct(three_points):
    # three out of p of them will need to collaborate to find the secret

    xs = [i[0] for i in three_points]
    ys = [i[1] for i in three_points]

    # create linear combinations that line up with x values such that x:comb
    # for xs = [1,3,5] we need 1:[3,5] , 3:[1,5], 5:[1,3] ie x not in comb

    pairs = lin_comb(xs)

    temp = deque(pairs)
    temp.rotate(2)
    pairs = list(temp)

    res = 0
    

    # interpolates the original curve with Lagrange polynomials
    # f(x) = sum of y_i*delta_i(x) for i in xs
    # delta_i(x) being formed from the ith pair (f,s) , delta_i(x) = -f/(i-f) * -s/(i-s)
    for i in range(3):
        idx = xs[i]
        f, s = pairs[i]

        delta = ((-f) / (idx - f)) * ((-s) / (idx - s))
        res += delta * ys[i]

    return round(res)

Step 4 - Run an example

In [23]:
m = 11

print("==- Shamirs Secret Sharing -==")
print("Generating shares...")
shares = shamir_generate(5, m)
print(shares)

three_shares = random.sample(shares, 3)
print("participants:", three_shares)

print("Reconstructing secret...")
secret = shamir_reconstruct(three_shares)
print("recovered secret:", secret)

if secret == m:
    print("Secret recovered successfully")

# needs modulus, prints and comments
# could be scaled up to n degrees, but constructing deltas would be harder


==- Shamirs Secret Sharing -==
Generating shares...
secret polynomial: Poly(41*x**2 + 50*x + 11, x, domain='ZZ')
secret: 11
points: [(48, 96875), (77, 246950), (99, 406802), (54, 122267), (85, 300486)]
[(48, 96875), (77, 246950), (99, 406802), (54, 122267), (85, 300486)]
participants: [(77, 246950), (99, 406802), (54, 122267)]
Reconstructing secret...
recovered secret: 11
Secret recovered successfully
